In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# simple data manipulation
import numpy  as np
import pandas as pd

# deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F

# results logging
import wandb

# progress bar
from   tqdm.notebook import tqdm, trange

# remove warnings (remove deprecated warnings)
import warnings
warnings.simplefilter('ignore')

# visualization of resultsa
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from   matplotlib.ticker import MaxNLocator
import seaborn           as sns
from scipy.interpolate import interp1d
# Graph Algorithms.
import networkx as nx

# Google Colab (many lines are removed)
import os
import zipfile
# from google.colab import drive
# from distutils.dir_util import copy_tree

# wheter we are using colab or not
COLAB: bool = False
if not COLAB and not os.path.exists('./data/simulations'): 
    os.chdir('..')

# Simulation Settings
from g6smart.sim_config import SimConfig
from g6smart.baseline.subband import cgc_algoritm
from g6smart.evaluation import rate as rate_metrics
import g6smart.evaluation  as evals

config = SimConfig(0)
config

## Simulations and Information

Thanks to the given scripts, we can load a group of generated simulations. They don't have any solutions (neither approximations).

In [ ]:
# Moung Google Drive Code
if COLAB:
    # drive.mount('/content/drive')

    # Move Simulations to avoid cluttering the drive folder
    # if not os.path.exists('/content/simulations'):
    #   os.mkdir('/content/simulations')

    # if list(os.listdir('/content/simulations')) == []:
    #   copy_tree('/content/drive/MyDrive/TFM/simulations', '/content/simulations')

    # unzip all simulations
    # print("Name of the already simulated data: \n", )
    for zip_file in os.listdir('/content/simulations'):
        if zip_file.endswith('.zip'):
            print(" ----> " + zip_file)
            with zipfile.ZipFile("/content/simulations/" + zip_file, 'r') as zip_ref:
                zip_ref.extractall('/content/simulations/')

    SIMULATIONS_PATH: str = "/content/simulations"
else:
    if not os.path.exists('./data/simulations'): os.mkdir('./data/simulations')
    for zip_file in os.listdir('data'):
        if zip_file.endswith('.zip'):
            print(" ----> " + zip_file)
            with zipfile.ZipFile("./data/" + zip_file, 'r') as zip_ref:
                zip_ref.extractall('./data/simulations')
    SIMULATIONS_PATH: str = "./data/simulations"

## Baselines

In [ ]:
cmg   = np.load(SIMULATIONS_PATH + '/Channel_matrix_gain.npy')
alloc = np.load(SIMULATIONS_PATH + '/sisa-allocation.npy').astype(int)

# get sample from all
n_sample = 12_000
cmg   = cmg[:n_sample]
alloc = alloc[:n_sample]

n_sample = cmg.shape[0]
K, N, _  = cmg.shape[1:]

shape    = lambda s: " x".join([f"{d:3d}" for d in s])
print(f"channel    matrix shape: {shape(cmg.shape)} \nallocation matrix shape: {shape(alloc.shape)}")

In [ ]:
srates, crates = [], []
for b in trange(n_sample):
    salloc = alloc[b]
    calloc = cgc_algoritm(cmg[b], 4)
    srate = rate_metrics.bit_rate(
        config, cmg[b], salloc, config.transmit_power
    )
    crate = rate_metrics.bit_rate(
        config, cmg[b], calloc, config.transmit_power
    )

    srates.append(srate)
    crates.append(crate)
srates = np.concat(srates, axis = 0)
crates = np.concat(crates, axis = 0)

In [ ]:
def add_cdf_plot(sample, label, color = None, ax = None, show_legend = True):
    ax = ax or plt.gca()
    
    # add CDF
    pos, cdf = evals.get_cdf(sample)
    ax.plot(pos, cdf, label = label, color = color)
    
    # cdf func
    cdf_func = interp1d(pos, cdf, bounds_error=False, fill_value=(0, 1))
    # add percentiles
    percentiles = [ 0.0005, 0.05, 0.25, 0.50]
    qs = np.quantile(sample, percentiles)
    styles = ['-', '-.', '--', ":"]
    for s, p, q in zip(styles, percentiles, qs):
        ax.plot(
            [q, q], [0, float(cdf_func(q))],
            linestyle=s, alpha=0.6, color = "black", 
            label=f'Q{p}' if show_legend else None
        )

plt.figure(figsize = (5.5, 5))

add_cdf_plot(srates, "SISA", "red", show_legend=True)
add_cdf_plot(crates, "CGC", "blue", show_legend=False)

# Sort legend: group quantiles at the end
handles, labels = plt.gca().get_legend_handles_labels()
normal = [(h, l) for h, l in zip(handles, labels) if not l.startswith("Q")]
quantiles = [(h, l) for h, l in zip(handles, labels) if l.startswith("Q")]
handles, labels = zip(*(normal + quantiles))
plt.legend(handles, labels)
plt.ylim(bottom=0)
plt.xlim(left=0)
plt.xlabel("Bit Rate (Mbps)")
plt.ylabel("CDF")
plt.title("Bit Rate's CDF")

plt.tight_layout()
plt.show()